# Analysis of Comic Book Reviews with usage of Deep NLP.

Dataset originates from the site https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home

The datasets were collected in late 2017 from goodreads.com, where only users' public shelves were scraped, i.e. everyone can see it on web without login. User IDs and review IDs are anonymized. 

Following papers needs to be mentioned for fair use:
1. Mengting Wan, Julian McAuley, "Item Recommendation on Monotonic Behavior Chains", in RecSys'18.
2. Mengting Wan, Rishabh Misra, Ndapa Nakashole, Julian McAuley, "Fine-Grained Spoiler Detection from Large-Scale Review Corpora", in ACL'19.

### Importing the data 

File is available with JSON extension, which is a standard data interchangeable format. To load it in Python we need to import json library.

In [2]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def load_data(file):
    reviews = []
    for line in  open(file, 'r'):
          reviews.append(json.loads(line))
    return reviews
filename = '/Users/user/Desktop/Python/ANN/NLP/goodreads_reviews_comics_graphic.json'
dataset = load_data(filename)
dataset_min = []
for line in dataset:
    dataset_min.append((line['review_text'],line['rating']))

#We are taking 1000 first rows to speed up the process
dataset = pd.DataFrame(dataset_min[:1000])
dataset.columns = ['Review', 'Rating']

### Cleaning the text 

First step for Natural Language Processing is of course cleaning the text. This is highly significat for model predictions. Following stages will be covered:
1. Deleting numbers, punctuation and trailing newlines,
2. Changing all uppercase letters to lowercase,
3. Removing as much meaningless words as possible based on 'stopwords' dataset,
4. Stemming - reducing inflected (or sometimes derived) words to their root form

Unfortunately English stopwords dataset contains word 'not' which is important for determining how good the review is, so we need to remove it from the stopwords dataset.

In [4]:
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

In [5]:
reviews_all = []
for i in range(0,len(dataset)):
    #1. Removing redundant signs
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.rstrip()
    #2. Changing uppercase letters
    review = review.lower()
    #3 & 4. Stopwords and stemming
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if word not in all_stopwords]
    review = ' '.join(review)
    reviews_all.append(review)

### Bag-of-Words 

Feature selection with BOW model.

In [6]:
cv = CountVectorizer()
X = cv.fit_transform(reviews_all).toarray()
y = dataset.iloc[:,-1].values
len(X[0])

6373

CountVectorizer class has an attribute of 'max_fatures' which indicates maximum number of columns a sparse matrix can have. Worth knowing is that this columns also point at meaningless words that won't influence the model itself. To reduce the number of columns we can manually set max_fatures to a certain number. 

In [7]:
max_f = 6000
cv = CountVectorizer(max_features = max_f)
X = cv.fit_transform(reviews_all).toarray()
y = dataset.iloc[:,-1].values

Encoding categorical data.

Usually for non-binary classification OneHotEncoder would be used, but here I want to transform multicategorical data to binary data. My goal is to predict not which rating the review will get but weather it will be negative or positive. That's why ratings from 0 to 2 are associated with 0, which means bad, and from 3 to 5 with 1, which is good.

In [8]:
y = np.where(y == 3, 1, np.where(y == 4, 1, np.where(y == 5, 1, 0)))

### Creating Artificial Neural Network

Splitting the dataset into training set and test set.

Since we have only categorical data in the dataset, we do not have to scale the features.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

For multicategorical data we would have several nodes as output, activation function would be softmax and loss would be categorical crossentropy. For binary output there is only one node needed, sigmoid function and binary crossentropy loss is also correct.

In [10]:
classifier= Sequential()
classifier.add(Dense(max_f/2, activation = 'relu'))
classifier.add(Dense(max_f/4, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 32, epochs = 100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
800/800 [==============================] - 5s 7ms/sample - loss: 0.5284 - acc: 0.7738
Epoch 2/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.1928 - acc: 0.9300
Epoch 3/100
800/800 [==============================] - 5s 7ms/sample - loss: 0.0403 - acc: 0.9900
Epoch 4/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0117 - acc: 0.9950
Epoch 5/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0071 - acc: 0.9975
Epoch 6/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0040 - acc: 0.9987
Epoch 7/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0042 - acc: 0.9987
Epoch 8/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0042 - acc: 0.9987
Epoch 9/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0033 - acc: 0.998

800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 79/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 80/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 81/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 82/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9975
Epoch 83/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0019 - acc: 0.9975
Epoch 84/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 85/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0017 - acc: 0.9987
Epoch 86/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9975
Epoch 87/100
800/800 [==============================] - 5s 6ms/sample - loss: 0.0018 - acc: 0.9987
Epoch 88/100
800/800 [=

In [11]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

In [12]:
acc = accuracy_score(y_test,y_pred)
acc

0.83

### Accuracy score is 83% which is indicating that the model is a good fit. Not too high value shows that model is in general not overfitted. 